In [1]:
# !pip install bertopic
from bertopic import BERTopic

# !pip install bertopic[visualization]
import pandas as pd 

# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# !pip install spacy
import spacy
# python -m spacy download en

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora

from pprint import pprint

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# !pip install pyLDAvis==2.1.2
import pyLDAvis
import pyLDAvis.gensim

[nltk_data] Downloading package stopwords to /Users/blam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/Users/blam/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [ ]:
#Import Reviews
reviews=pd.read_json("/Users/blam/Documents/Datascience capstone/yelp_academic_dataset_review.json", lines=True)
reviews

In [ ]:
#Import Non-Food Reviews
reviews=pd.read_csv("/Users/blam/Documents/Datascience capstone/non_food_merged.csv")
reviews

In [ ]:
business=pd.read_json("/Users/blam/Documents/Datascience capstone/yelp_academic_dataset_business.json", lines=True)
business

In [ ]:
business_categories = business[['business_id','categories']]
business_categories[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35]] = (business_categories['categories'].str.split(',', expand=True))
business_categories


In [ ]:
business_nonfood = business_categories.loc[~(business_categories[0].isin(['Restaurants','Food','Printing Services','Pest Control'])) & ~(business_categories[1].isin([' Restaurants',' Food'])) & ~(business_categories[2].isin([' Restaurants',' Food'])) & ~(business_categories[3].isin([' Restaurants',' Food'])) & ~(business_categories[4].isin([' Restaurants',' Food'])) & ~(business_categories[5].isin([' Restaurants',' Food'])) & ~(business_categories[6].isin([' Restaurants',' Food']))]
print(len(business_nonfood[0]))
business_nonfood.head(50)

In [3]:
reviews_text = reviews[['business_id','text']]
reviews_text

,business_id,text
0,n_0UpQx1hsNbnPUSlodU8w,This has become my go to place for shoes. I a...
1,n_0UpQx1hsNbnPUSlodU8w,"Oh, I do enjoy Famous Footwear. \n\nThe occasi..."
2,n_0UpQx1hsNbnPUSlodU8w,Ordered shoes online it clearly says free retu...
3,n_0UpQx1hsNbnPUSlodU8w,Poor customer service will never be shopping a...
4,n_0UpQx1hsNbnPUSlodU8w,Found a great deal on a pair of Nike running s...
...,...,...
665144,jV_XOycEzSlTx-65W906pg,It is with my greatest pleasure to say that I'...
665145,jV_XOycEzSlTx-65W906pg,I went there to get my nose piercing replaced....
665146,jV_XOycEzSlTx-65W906pg,Fantastic tattoo artist. Friendly and talkativ...
665147,jV_XOycEzSlTx-65W906pg,"Please note, this studio has relocated to Apol..."


In [ ]:
# reviews_nofood = pd.merge(reviews_text, business_nonfood, how='inner', on = 'business_id')
# reviews_nofood

In [101]:
# Text preprocessing functions
stop_words = stopwords.words('english')
stop_words.extend(['from', 'to', 'how', 'they', 'very','many'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [105]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(reviews_text, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[reviews_text], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [106]:
# Remove Stop Words
data_words_nostops = remove_stopwords(reviews_nofood_text)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['take', 'lot', 'spin', 'class', 'year', 'compare', 'class', 'body', 'cycle', 'nice', 'clean', 'space', 'amazing', 'bike', 'welcome', 'motivate', 'instructor', 'class', 'top', 'notch', 'work', 'struggle', 'online', 'scheduling', 'system', 'make', 'easy', 'plan', 'ahead', 'need', 'line', 'way', 'advanced', 'gym', 'make', 'way', 'write', 'review', 'give', 'russell', 'owner', 'body', 'cycle', 'shout', 'russell', 'passion', 'fitness', 'cycling', 'evident', 'desire', 'client', 'succeed', 'always', 'drop', 'class', 'check', 'provide', 'encouragement', 'open', 'idea', 'recommendation', 'russell', 'always', 'wear', 'smile', 'face', 'even', 'kick', 'butt', 'class']]


In [107]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 5), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 3), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 2), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1)]]


In [108]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=6, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [109]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.018*"go" + 0.017*"get" + 0.015*"great" + 0.013*"time" + 0.013*"nail" + '
  '0.012*"hair" + 0.012*"make" + 0.011*"good" + 0.011*"place" + 0.010*"look"'),
 (1,
  '0.040*"room" + 0.020*"stay" + 0.019*"hotel" + 0.014*"clean" + 0.010*"nice" '
  '+ 0.009*"get" + 0.009*"staff" + 0.009*"night" + 0.008*"place" + '
  '0.008*"bed"'),
 (2,
  '0.018*"get" + 0.017*"say" + 0.017*"go" + 0.016*"tell" + 0.016*"call" + '
  '0.012*"time" + 0.010*"ask" + 0.009*"take" + 0.009*"never" + 0.008*"back"'),
 (3,
  '0.012*"store" + 0.011*"go" + 0.010*"great" + 0.009*"place" + 0.009*"get" + '
  '0.009*"see" + 0.008*"time" + 0.008*"find" + 0.007*"good" + 0.007*"well"'),
 (4,
  '0.029*"car" + 0.018*"service" + 0.014*"work" + 0.014*"get" + 0.010*"need" + '
  '0.010*"go" + 0.010*"time" + 0.009*"take" + 0.009*"price" + 0.009*"new"'),
 (5,
  '0.019*"food" + 0.018*"place" + 0.018*"good" + 0.015*"great" + 0.014*"bar" + '
  '0.013*"drink" + 0.013*"go" + 0.012*"get" + 0.009*"time" + 0.008*"come"')]


In [110]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score. Higher the topic coherence, the topic is more human interpretable.
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.34421878743206

Coherence Score:  0.4075701427655356


In [111]:
def tsnescatterplot(model, word, list_names):
    """ Plot in seaborn the results from the t-SNE dimensionality reduction algorithm of the vectors of a query word,
    its list of most similar words, and a list of words.
    """
    arrays = np.empty((0, 300), dtype='f')
    word_labels = [word]
    color_list  = ['red']

    # adds the vector of the query word
    arrays = np.append(arrays, model.wv.__getitem__([word]), axis=0)
    
    # gets list of most similar words
    close_words = model.wv.most_similar([word])
    
    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)
    
    # adds the vector for each of the words from list_names to the array
    for wrd in list_names:
        wrd_vector = model.wv.__getitem__([wrd])
        word_labels.append(wrd)
        color_list.append('green')
        arrays = np.append(arrays, wrd_vector, axis=0)
        
    # Reduces the dimensionality from 300 to 50 dimensions with PCA
    reduc = PCA(n_components=20).fit_transform(arrays)
    
    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)
    
    Y = TSNE(n_components=2, random_state=0, perplexity=15).fit_transform(reduc)
    
    # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    
    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                 }
                    )
    
    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
         p1.text(df["x"][line],
                 df['y'][line],
                 '  ' + df["words"][line].title(),
                 horizontalalignment='left',
                 verticalalignment='bottom', size='medium',
                 color=df['color'][line],
                 weight='normal'
                ).set_size(15)

    
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
            
    plt.title('t-SNE visualization for {}'.format(word.title()))

In [112]:
tsnescatterplot(w2v_model, 'java',[t[0] for t in w2v_model.wv.most_similar(positive=["java"], topn=20)][10:])

NameError: name 'w2v_model' is not defined

In [114]:
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=id2word)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

ValidationError: 
 * Not all rows (distributions) in topic_term_dists sum to 1.

In [4]:
reviews_text['text'].to_list()

["This has become my go to place for shoes.  I am a rewards member and they often send me coupons and notices about their sales.  The best sale they have is the BOGO 1/2 off plus 15% off.  That is a great time to stock up on shoes especially if you have a coupon on top of that.   \n\nOther reviewers have complained about the return policy and I must say that I have never attempted to return anything at this store so I can't really comment on that.  I guess that is just something to be aware of before making a purchase.  \n\nThis store also may not have your size for the shoe you want but they can usually order it for you and have it delivered to the store or your house.  I haven't ordered a shoe this way but employees have alerted me to this service. \n\nI'd recommend going here for a good price on shoes.  They have men's and women's and kids shoes as well as casual, athletic, and formal shoes.  If you don't find anything here, there is always DSW next door.",
 "Oh, I do enjoy Famous F

In [7]:
# create model 
model = BERTopic(verbose=True, nr_topics= 5)
 
#convert to list 
docs = reviews_text['text'].to_list()
 
topics, probabilities = model.fit_transform(docs)



: 

: 

In [ ]:
# Select top topics
print(model.get_topic_freq().head(11))
print(model.get_topic(6))
model.visualize_topics()

In [ ]:
model.visualize_barchart(top_n_topics=8, n_words=20, height=500)

In [ ]:
model.visualize_hierarchy(height=1000)